### Mustafa Kutay Yabas - EC581 - 29.10.2016
# Assignment #3

We will test two trading strategies based on return runs.
- Trend Following Strategy
    - Buy after n days of positive return
    - Sell after m days of negative return
- Mean Reversion Strategy
    - Buy after n days of negative return
    - Sell after m days of positive return

### Steps
1. Apply two different trading algo for these strategies in quantstrat
2. Apply these strategies to BIST100 index data
3. For each strategy, optimize n and m using a grid search over set 1,2,...,10
4. Compare optimized versions of these two strategies
5. Is it better to be a trend-follower or a contrarian in BIST

# Results

You may find the R codes below. At the top of the document I want to talk about the problems I have encountered with quantsrat and how I have overcame those.

1. To analyze the consecutive days I wrote a custom function. I created signal columns with that function and based the quantstrat rules over those signals.
2. However I couldn't use the same function (consecutive_days) for grid analysis (add.distribution).
3. Thats why I looped through variables for optimization process. I didn't save all the variables but just the end_equity for faster execution. So make the performance comparison according to end equity.
4. Then I repeated the same process for mean reversion strategy.

## Comparison of Results
### Trend Following Strategy
positive days: 2  
negative days: 3  
end equity: 185773.68  

Trend following strategy goes long after 2 positive days and close the trade after 3 negative days or goes short after 3 negative days and close the trade after 2 positive days.

Trend following is better than mean reversion and buy&hold strategies.

### Mean Reversion Strategy
positive days: 9  
negative days: 1  
end equity: 144742.20  

Mean reversion strategy goes short after 9 positive days and close the trade after 1 negative day or goes long after 1 negative day and close the trade after 9 positive days.
 
### Buy&Hold Strategy
end equity: 137234.22

Both Trend following and Mean reversion strategy have potential to beat the buy&hold strategy.

## Discussion
In this optimization process positive day and negative day variables kept same for both short and long positions. Thus this demonstration is not a flawless one. A better optimization strategy should seperate long and short positions.

As I can't use the custom signal function with quantstrat's add.distribution function, I don't have an extensive comparison of the optimized strategies. Drawdown, sharpe ratio etc. should also be considered for a better benchmark. It could also have been done by returning the order book from the test_strategy function but it would take a lot more effort.

Money management should be considered for real world trading. This backtest just buys or sells 1 lot of stock for each trade. Different rules to exit/close the trades may be used. Preset limits or trailing stops may yield better end equities than closing the trade after n days of a trend.

The optimization process most probably affected by sharp market movements. For better optimization market data should be divided in to two as control and treatment groups.

__Despite all the drawbacks, both trend following and mean reversion strategies are promising over buy&hold strategy.__

# R Codes

In [14]:
# load libraries
library(quantstrat)
library(Quandl)

In [15]:
# define instruments
currency("USD")
stock("BIST", currency="USD", multiplier=1)

# get data
date_from = "2005-08-01"
date_to = "2016-05-25"

BIST<-Quandl("GOOG/INDEXIST_XU100", type="xts", start_date = date_from, end_date = date_to)
BIST<-na.omit(BIST)
BIST<-xts(coredata(BIST), as.POSIXct(time(BIST)))

BIST_back<-BIST #backup

[1] "USD"

[1] "BIST"

In [16]:
consecutive_days<-function(days_pos,days_neg, stock, posneg = TRUE) {
    #days_pos <- 4
    #days_neg <- 4

    #n_day_signals <- data.frame(positive = logical(length(time(stock))), negative = logical(length(time(stock))))
    n_day_signals <- data.frame(sigcol = logical(length(time(stock))))
    n_day_signals <- xts( n_day_signals, as.POSIXct(time(stock)) )
    n_day_signals[1,1] <- NA

    #Signal <- xts(c("Positive", "Negative"), as.POSIXct(time(BIST)))

    sign_counter <- 1
    sign_last <- -1
    for (i in 2:length(time(stock))) {

        sign_temp <- sign( as.numeric ( as.numeric( stock[i,4]) - as.numeric( stock[i-1,4]) ) )

        if (sign_temp == sign_last) {
            sign_counter <- sign_counter + 1
        } else {
            sign_counter <- 1
            sign_last <- sign_temp
        }

        if (posneg) {
            
            if (sign_counter == days_pos && sign_last == 1) {
                n_day_signals[i,1] <- TRUE
            } else {
                n_day_signals[i,1] <- NA
            }
            
        } else {

            if (sign_counter == days_neg && sign_last == -1) {
                n_day_signals[i,1] <- TRUE
            } else {
                n_day_signals[i,1] <- NA
            }
        }
    }
    
    if (posneg == TRUE) {
        return( n_day_signals$sigcol) 
    } else {
        return(n_day_signals$sigcol)
    }

}

In [75]:
test_strategy<-function(days_pos, days_neg) {
BIST <- BIST_back
# define strategy component names
portfolio_name = "investiphi"

#strategy_trend = "trend_following"
strategy.st = "consecutive_days"

#account_trend = "account_trend"
account_name = "account_name"

# remove if defined before
rm.strat(portfolio_name)

#rm.strat(strategy_trend)
rm.strat(strategy.st)

#rm.strat(account_trend)
rm.strat(account_name)

# create .blotter and .strategy environments
.blotter<-new.env()
.strategy<-new.env()

# init portfolio and accoiunt in .blotter
init_eq <- 100000 # 100k
init_date <- as.character(as.Date(date_from) - 1)

initPortf(portfolio_name, symbols="BIST", initDate=init_date, currency="USD")

#initAcct(account_trend, portfolios=portfolio_name, initDate=init_date, currency="USD", initEq = init_eq)
initAcct(account_name, portfolios=portfolio_name, initDate=init_date, currency="USD", initEq = init_eq)

initOrders(portfolio_name, initDate=init_date)

# init strategies
#strategy(strategy_trend, store=TRUE)
strategy(strategy.st, store=TRUE)

add.signal(strategy.st, name="consecutive_days",
           arguments = list(days_pos = days_pos, days_neg = days_neg, stock=BIST, posneg=TRUE),
           label="bull"
          )

add.signal(strategy.st, name="consecutive_days",
           arguments = list(days_pos = days_pos, days_neg = days_neg, stock=BIST, posneg=FALSE),
           label="bear"
          )

order_qty = 1
add.rule(strategy.st, name='ruleSignal',
         arguments=list(sigcol='sigcol.bear',
                        sigval=1,
                        orderside='short',
                        ordertype='market',
                        orderqty=-order_qty,
                        TxnFees=0,
                        replace=FALSE),
         type='enter',
         label='EnterShort'
        )

add.rule(strategy.st, name='ruleSignal',
         arguments=list(sigcol='sigcol.bull',
                        sigval=1,
                        orderside='long',
                        ordertype='market',
                        orderqty='all',
                        TxnFees=0,
                        replace=TRUE),
         type='exit',
         label='Exit2Long'
        )

add.rule(strategy.st, name='ruleSignal',
         arguments=list(sigcol='sigcol.bull',
                        sigval=TRUE,
                        orderside='long',
                        ordertype='market',
                        orderqty=order_qty,
                        TxnFees=0,
                        replace=FALSE),
         type='enter',
         label='EnterLong'
        )

add.rule(strategy.st, name='ruleSignal',
         arguments=list(sigcol='sigcol.bear',
                        sigval=TRUE,
                        orderside='short',
                        ordertype='market',
                        orderqty='all',
                        TxnFees=0,
                        replace=TRUE),
         type='exit',
         label='Exit2Short'
        )

applyStrategy(strategy.st, portfolio_name)
updatePortf(portfolio_name)
updateAcct(account_name)
updateEndEq(account_name)

#acc <- get('account.account_name', envir=.blotter)
acc = getAccount(account_name)
returns = list("endeq" = getEndEq(account_name, date_to), "account" = acc)
return(returns)
}

In [76]:
results = matrix(nrow=100, ncol=3)
for (days_pos in 1:10) {
    for(days_neg in 1:10) {
        i = (days_pos-1)*10 + days_neg
        results[i,1] = days_pos
        results[i,2] = days_neg
        returns = test_strategy(days_pos, days_neg)
        results[i,3] = returns$endeq
        saveRDS( returns$account, file = paste("rds-trend/", days_pos, "-", days_neg,".rds", sep = "") )
        assign(paste("account_data.",days_pos,".",days_neg,sep=""), returns$account)
    }
}
saveRDS( results, file = "rds-trend/results.rds")

[1] "2005-08-04 03:00:00 BIST 1 @ 30123.59"
[1] "2005-08-08 03:00:00 BIST -1 @ 29924.83"
[1] "2005-08-17 03:00:00 BIST 1 @ 27906.28"
[1] "2005-08-18 03:00:00 BIST -1 @ 28000"
[1] "2005-08-19 03:00:00 BIST 1 @ 28456.86"
[1] "2005-08-31 03:00:00 BIST -1 @ 30908.02"
[1] "2005-09-01 03:00:00 BIST 1 @ 31947.95"
[1] "2005-09-05 03:00:00 BIST -1 @ 31879.45"
[1] "2005-09-06 03:00:00 BIST 1 @ 31832.5"
[1] "2005-09-07 03:00:00 BIST -1 @ 31385.01"
[1] "2005-09-09 03:00:00 BIST 1 @ 32202.66"
[1] "2005-09-14 03:00:00 BIST -1 @ 32632.44"
[1] "2005-09-15 03:00:00 BIST 1 @ 33271.11"
[1] "2005-09-20 03:00:00 BIST -1 @ 33719.3"
[1] "2005-09-21 03:00:00 BIST 1 @ 33863.82"
[1] "2005-09-23 03:00:00 BIST -1 @ 33195.6"
[1] "2005-09-27 03:00:00 BIST 1 @ 33319.18"
[1] "2005-09-28 03:00:00 BIST -1 @ 32744.95"
[1] "2005-09-30 03:00:00 BIST 1 @ 33333.23"
[1] "2005-10-03 03:00:00 BIST -1 @ 34300.9"
[1] "2005-10-04 03:00:00 BIST 1 @ 35624.79"
[1] "2005-10-06 03:00:00 BIST -1 @ 33510.39"
[1] "2005-10-11 03:00:00 BIS

In [39]:
print(results)
# positive days, negative days, end balance

       [,1] [,2]      [,3] [,4]
  [1,]    1    1 147782.00   NA
  [2,]    1    2 153579.00   NA
  [3,]    1    3 164743.97   NA
  [4,]    1    4 153019.39   NA
  [5,]    1    5 154354.82   NA
  [6,]    1    6 147417.46   NA
  [7,]    1    7 138993.11   NA
  [8,]    1    8 141846.75   NA
  [9,]    1    9 139613.38   NA
 [10,]    1   10 137559.16   NA
 [11,]    2    1 105064.16   NA
 [12,]    2    2 155410.65   NA
 [13,]    2    3 185773.68   NA
 [14,]    2    4 148293.22   NA
 [15,]    2    5 150977.49   NA
 [16,]    2    6 149849.08   NA
 [17,]    2    7 145654.01   NA
 [18,]    2    8 152944.07   NA
 [19,]    2    9 141582.09   NA
 [20,]    2   10 139527.87   NA
 [21,]    3    1  76974.20   NA
 [22,]    3    2  99279.04   NA
 [23,]    3    3 160367.37   NA
 [24,]    3    4 167518.31   NA
 [25,]    3    5 157007.25   NA
 [26,]    3    6 161471.04   NA
 [27,]    3    7 156192.93   NA
 [28,]    3    8 161317.69   NA
 [29,]    3    9 141737.34   NA
 [30,]    3   10 139683.12   NA
 [31,]  

In [77]:
#calculate buy&hold end equity
100000*(as.numeric(BIST[length(time(BIST))]$Close) - as.numeric(BIST[1]$Close))/as.numeric(BIST[1]$Close)

[1] 137234.2

In [80]:
# just copy and paste the function for adapting to mean reversion strategy
# and change the rule variables accordingly
test_strategy_mean<-function(days_pos, days_neg) { #mean reversion
BIST <- BIST_back
# define strategy component names
portfolio_name = "investiphi"

#strategy_trend = "trend_following"
strategy.st = "consecutive_days"

#account_trend = "account_trend"
account_name = "account_name"

# remove if defined before
rm.strat(portfolio_name)

#rm.strat(strategy_trend)
rm.strat(strategy.st)

#rm.strat(account_trend)
rm.strat(account_name)

# create .blotter and .strategy environments
.blotter<-new.env()
.strategy<-new.env()

# init portfolio and accoiunt in .blotter
init_eq <- 100000 # 100k
init_date <- as.character(as.Date(date_from) - 1)

initPortf(portfolio_name, symbols="BIST", initDate=init_date, currency="USD")

#initAcct(account_trend, portfolios=portfolio_name, initDate=init_date, currency="USD", initEq = init_eq)
initAcct(account_name, portfolios=portfolio_name, initDate=init_date, currency="USD", initEq = init_eq)

initOrders(portfolio_name, initDate=init_date)

# init strategies
#strategy(strategy_trend, store=TRUE)
strategy(strategy.st, store=TRUE)

add.signal(strategy.st, name="consecutive_days",
           arguments = list(days_pos = days_pos, days_neg = days_neg, stock=BIST, posneg=TRUE),
           label="bull"
          )

add.signal(strategy.st, name="consecutive_days",
           arguments = list(days_pos = days_pos, days_neg = days_neg, stock=BIST, posneg=FALSE),
           label="bear"
          )

order_qty = 1
add.rule(strategy.st, name='ruleSignal',
         arguments=list(sigcol='sigcol.bear',
                        sigval=1,
                        orderside='long',
                        ordertype='market',
                        orderqty=order_qty,
                        TxnFees=0,
                        replace=FALSE),
         type='enter',
         label='EnterLong'
        )

add.rule(strategy.st, name='ruleSignal',
         arguments=list(sigcol='sigcol.bull',
                        sigval=1,
                        orderside='short',
                        ordertype='market',
                        orderqty='all',
                        TxnFees=0,
                        replace=TRUE),
         type='exit',
         label='Exit2Short'
        )

add.rule(strategy.st, name='ruleSignal',
         arguments=list(sigcol='sigcol.bull',
                        sigval=TRUE,
                        orderside='short',
                        ordertype='market',
                        orderqty=-order_qty,
                        TxnFees=0,
                        replace=FALSE),
         type='enter',
         label='EnterShort'
        )

add.rule(strategy.st, name='ruleSignal',
         arguments=list(sigcol='sigcol.bear',
                        sigval=TRUE,
                        orderside='long',
                        ordertype='market',
                        orderqty='all',
                        TxnFees=0,
                        replace=TRUE),
         type='exit',
         label='Exit2Long'
        )

applyStrategy(strategy.st, portfolio_name)
updatePortf(portfolio_name)
updateAcct(account_name)
updateEndEq(account_name)
    
acc = getAccount(account_name)
returns = list("endeq" = getEndEq(account_name, date_to), "account" = acc)
return(returns)
}

In [81]:
results = matrix(nrow=100, ncol=3)
for (days_pos in 1:10) {
    for(days_neg in 1:10) {
        i = (days_pos-1)*10 + days_neg
        results[i,1] = days_pos
        results[i,2] = days_neg
        returns = test_strategy_mean(days_pos, days_neg)
        results[i,3] = returns$endeq
        saveRDS( returns$account, file = paste("rds-mean/", days_pos, "-", days_neg,".rds", sep = "") )
        assign(paste("account_data.",days_pos,".",days_neg,sep=""), returns$account)
    }
}
saveRDS( results, file = "rds-mean/results.rds")

[1] "2005-08-04 03:00:00 BIST -1 @ 30123.59"
[1] "2005-08-08 03:00:00 BIST 1 @ 29924.83"
[1] "2005-08-17 03:00:00 BIST -1 @ 27906.28"
[1] "2005-08-18 03:00:00 BIST 1 @ 28000"
[1] "2005-08-19 03:00:00 BIST -1 @ 28456.86"
[1] "2005-08-31 03:00:00 BIST 1 @ 30908.02"
[1] "2005-09-01 03:00:00 BIST -1 @ 31947.95"
[1] "2005-09-05 03:00:00 BIST 1 @ 31879.45"
[1] "2005-09-06 03:00:00 BIST -1 @ 31832.5"
[1] "2005-09-07 03:00:00 BIST 1 @ 31385.01"
[1] "2005-09-09 03:00:00 BIST -1 @ 32202.66"
[1] "2005-09-14 03:00:00 BIST 1 @ 32632.44"
[1] "2005-09-15 03:00:00 BIST -1 @ 33271.11"
[1] "2005-09-20 03:00:00 BIST 1 @ 33719.3"
[1] "2005-09-21 03:00:00 BIST -1 @ 33863.82"
[1] "2005-09-23 03:00:00 BIST 1 @ 33195.6"
[1] "2005-09-27 03:00:00 BIST -1 @ 33319.18"
[1] "2005-09-28 03:00:00 BIST 1 @ 32744.95"
[1] "2005-09-30 03:00:00 BIST -1 @ 33333.23"
[1] "2005-10-03 03:00:00 BIST 1 @ 34300.9"
[1] "2005-10-04 03:00:00 BIST -1 @ 35624.79"
[1] "2005-10-06 03:00:00 BIST 1 @ 33510.39"
[1] "2005-10-11 03:00:00 BIS

In [82]:
print(results)
# positive days, negative days, end balance

       [,1] [,2]      [,3]
  [1,]    1    1  52218.00
  [2,]    1    2  46421.00
  [3,]    1    3  35256.03
  [4,]    1    4  46980.61
  [5,]    1    5  45645.18
  [6,]    1    6  52582.54
  [7,]    1    7  61006.89
  [8,]    1    8  58153.25
  [9,]    1    9  60386.62
 [10,]    1   10  62440.84
 [11,]    2    1  94935.84
 [12,]    2    2  44589.35
 [13,]    2    3  14226.32
 [14,]    2    4  51706.78
 [15,]    2    5  49022.51
 [16,]    2    6  50150.92
 [17,]    2    7  54345.99
 [18,]    2    8  47055.93
 [19,]    2    9  58417.91
 [20,]    2   10  60472.13
 [21,]    3    1 123025.80
 [22,]    3    2 100720.96
 [23,]    3    3  39632.63
 [24,]    3    4  32481.69
 [25,]    3    5  42992.75
 [26,]    3    6  38528.96
 [27,]    3    7  43807.07
 [28,]    3    8  38682.31
 [29,]    3    9  58262.66
 [30,]    3   10  60316.88
 [31,]    4    1 128937.87
 [32,]    4    2  95500.70
 [33,]    4    3  41569.82
 [34,]    4    4  57370.40
 [35,]    4    5  39239.64
 [36,]    4    6  32269.86
 